In [1]:
import ROOT as rt

Welcome to JupyROOT 6.14/04


In [2]:
import argparse
import os
import sys

def setUnfoldBkgs(unfold_class, hfile_path, syst_name, isSys, nthSys, nTotSys, year, DYFake = False):

    if DYFake is False:
        unfold_class.subBkgs("Pt", hfile_path, "DYJetsToTauTau",       isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "DYJets10to50ToTauTau", isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "TTLL_powheg",          isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "WW_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "WZ_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "ZZ_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Pt", hfile_path, "WJets_MG",             isSys, syst_name, nTotSys, nthSys, "detector_level")

        unfold_class.subBkgs("Mass", hfile_path, "DYJetsToTauTau",       isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "DYJets10to50ToTauTau", isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "TTLL_powheg",          isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "WW_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "WZ_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "ZZ_pythia",            isSys, syst_name, nTotSys, nthSys, "detector_level")
        unfold_class.subBkgs("Mass", hfile_path, "WJets_MG",             isSys, syst_name, nTotSys, nthSys, "detector_level")
        
    else:
        unfold_class.subBkgs("Pt", hfile_path, "DYJets",       isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
        unfold_class.subBkgs("Mass", hfile_path, "DYJets",       isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
        if year == "2016":
            unfold_class.subBkgs("Pt", hfile_path, "DYJets10to50", isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
            unfold_class.subBkgs("Mass", hfile_path, "DYJets10to50", isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
        else:
            unfold_class.subBkgs("Pt", hfile_path, "DYJets10to50_MG", isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")
            unfold_class.subBkgs("Mass", hfile_path, "DYJets10to50_MG", isSys, syst_name, nTotSys, nthSys, "detector_level_DY_Fake")

In [3]:
def doISRAnalysis(year, channel, doSys):
    
    phase_space_detector = 'fiducial_phase_pre_FSR_dRp1'
    FSR_dR_detector      = 'dressed_dRp1'
    phase_space_fsr      = 'full_phase_dRp1'
    FSR_dR_fsr           = 'dRp1_pre_fsr'

    # set output directory
    outputDirectory = 'output/'+year+'/' + channel + "/"
    inputfhisttxtName = outputDirectory + "fhist.txt"

    # make output directory
    if not os.path.exists( outputDirectory ):
        os.makedirs( outputDirectory )

    # read text file including root file path for unfolding
    fOutTxtCheck = open( inputfhisttxtName,'r')
    unfoldInputList = {}

    for line in fOutTxtCheck:
            modifiedLine = line.lstrip(' ').rstrip(' ').rstrip('\n')
            if modifiedLine.split()[1] == "matrix":
                    unfoldInputList['matrix'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "matrix_pdferr":
                    unfoldInputList['matrix_pdferr'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "matrix_closure":
                    unfoldInputList['matrix_closure'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "fsr_matrix_closure":
                    unfoldInputList['fsr_matrix_closure'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "matrix_alt":
                    unfoldInputList['matrix_alt'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "fsr_matrix":
                    unfoldInputList['fsr_matrix'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "fsr_matrix_alt":
                    unfoldInputList['fsr_matrix_alt'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "fsr_photos_matrix":
                    unfoldInputList['fsr_photos_matrix'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "fsr_pythia_matrix":
                    unfoldInputList['fsr_pythia_matrix'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "gen_hist":
                    unfoldInputList['gen_hist'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "gen_hist_closure":
                    unfoldInputList['gen_hist_closure'] = modifiedLine.split()[2]

            if modifiedLine.split()[1] == "hist":
                    unfoldInputList['hist'] = modifiedLine.split()[2]

    print(unfoldInputList)

    import pyScripts.unfoldUtil as unfoldutil
    import pyScripts.drawUtil as drawutil

    DetectorUnfold = 0;
    FSRUnfold = 1;

    bias = 1.0

    # create unfold class                                                                    # regularization mode
    unfoldClass = rt.ISRUnfold(channel, unfoldInputList['hist'], unfoldInputList['gen_hist'], False, int(year), int(0))
    unfoldClass.setOutputBaseDir(outputDirectory)
    unfoldClass.setBias(bias)
    # set response matrix
    unfoldClass.SetNomTUnfoldDensity("Pt",  unfoldInputList['matrix'], phase_space_detector, FSR_dR_detector, unfoldInputList['matrix_closure'])
    unfoldClass.SetNomTUnfoldDensity("Mass",unfoldInputList['matrix'], phase_space_detector, FSR_dR_detector, unfoldInputList['matrix_closure'])

    unfoldClass.doClosureTest(DetectorUnfold, unfoldInputList['matrix_closure'], "detector_level")

    # set unfolding input histogram
    unfoldClass.setInput("Pt",   unfoldInputList['hist'], False, "nominal", 0, bias, "detector_level")
    unfoldClass.setInput("Mass", unfoldInputList['hist'], False, "nominal", 0, bias, "detector_level")
    setUnfoldBkgs(unfoldClass, unfoldInputList['hist'],   "nominal", False, 0, -1, year)
    setUnfoldBkgs(unfoldClass, unfoldInputList['matrix'], "nominal", False, 0, -1, year, True)

    # set systematic response matrix and input histograms
    if channel == "electron" : sysDict = {"PU": 2, "trgSF": 2, "recoSF": 2, "IdSF": 2, "AlphaS": 2, "Scale": 6, "lepMom": 2, "Alt": 1, "Stat": 500} # "Stat"

    if channel == "muon" :     sysDict = {"PU": 2, "trgSF": 2, "IsoSF": 2, "IdSF": 2, "AlphaS": 2, "Scale": 6, "Alt": 1, "Stat": 500}
    if doSys == True:
         
        for sysName, nSys in sysDict.items():
            for nthSys in range(0,nSys):
    
                print("sysName: " + sysName + " nthSys: " + str(nthSys) + " #####################################################")

                if sysName == "Alt":
                    unfoldClass.setSysTUnfoldDensity("Pt",  unfoldInputList['matrix'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
                    unfoldClass.setSysTUnfoldDensity("Mass",unfoldInputList['matrix'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
                elif sysName == "PDFerror":
                    unfoldClass.setSysTUnfoldDensity("Pt",  unfoldInputList['matrix_pdferr'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
                    unfoldClass.setSysTUnfoldDensity("Mass",unfoldInputList['matrix_pdferr'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
                else :
                    # set systematic response matrix
                    unfoldClass.setSysTUnfoldDensity("Pt",  unfoldInputList['matrix'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)
                    unfoldClass.setSysTUnfoldDensity("Mass",unfoldInputList['matrix'],  sysName, nSys, nthSys, phase_space_detector, FSR_dR_detector)

                if sysName == "unfoldBias": bias = 0.95

                # set systematic input histograms
                unfoldClass.setInput("Pt",   unfoldInputList['hist'], True, sysName, nthSys, bias, "detector_level")
                unfoldClass.setInput("Mass", unfoldInputList['hist'], True, sysName, nthSys, bias, "detector_level")

                # set systematic background histograms
                if sysName is not "Stat":
                    if sysName == "PDFerror":
                        setUnfoldBkgs(unfoldClass, unfoldInputList['hist'],          sysName, True, nthSys, nSys, year)
                        setUnfoldBkgs(unfoldClass, unfoldInputList['matrix_pdferr'], sysName, True, nthSys, nSys, year, True)
                    else:
                        setUnfoldBkgs(unfoldClass, unfoldInputList['hist'],   sysName, True, nthSys, nSys, year)
                        setUnfoldBkgs(unfoldClass, unfoldInputList['matrix'], sysName, True, nthSys, nSys, year, True)

    
    # unfold
    unfoldClass.doISRUnfold(DetectorUnfold, doSys)

    # set QED FSR unfolding response matrix and input
    unfoldClass.setNomFSRTUnfoldDensity("Pt",    unfoldInputList['fsr_matrix'], phase_space_fsr, FSR_dR_fsr, unfoldInputList['fsr_matrix_closure'])
    unfoldClass.setNomFSRTUnfoldDensity("Mass",  unfoldInputList['fsr_matrix'], phase_space_fsr, FSR_dR_fsr, unfoldInputList['fsr_matrix_closure'])
    unfoldClass.setFSRUnfoldInput(unfoldInputList['fsr_matrix'], False, "", -1, phase_space_fsr)

    unfoldClass.doClosureTest(FSRUnfold, unfoldInputList['fsr_matrix_closure'], "fiducial_phase_dRp1")

    if doSys == True:
        # systematic from detector unfolding
        for sysName, nSys in sysDict.items():
            for nthSys in range(0,nSys):

                if sysName == "Alt":
                    unfoldClass.setSysFSRTUnfoldDensity("Pt",   unfoldInputList['fsr_pythia_matrix'], sysName, nSys, nthSys, phase_space_fsr, FSR_dR_fsr)
                    unfoldClass.setSysFSRTUnfoldDensity("Mass", unfoldInputList['fsr_pythia_matrix'], sysName, nSys, nthSys, phase_space_fsr, FSR_dR_fsr)

                else :
                    unfoldClass.setSysFSRTUnfoldDensity("Pt",   unfoldInputList['fsr_matrix'], sysName, nSys, nthSys, phase_space_fsr, FSR_dR_fsr)
                    unfoldClass.setSysFSRTUnfoldDensity("Mass", unfoldInputList['fsr_matrix'], sysName, nSys, nthSys, phase_space_fsr, FSR_dR_fsr)

                unfoldClass.setFSRUnfoldInput(unfoldInputList['fsr_matrix'], True, sysName, nthSys)

        # QED FSR ststematic
        sysDict["QED_FSR"] = 2

        # QED FSR ststematic
        unfoldClass.setSysFSRTUnfoldDensity("Pt",   unfoldInputList['fsr_photos_matrix'], "QED_FSR", 2, 0, phase_space_fsr, FSR_dR_fsr)
        unfoldClass.setSysFSRTUnfoldDensity("Mass", unfoldInputList['fsr_photos_matrix'], "QED_FSR", 2, 0, phase_space_fsr, FSR_dR_fsr)

        unfoldClass.setFSRUnfoldInput(unfoldInputList['fsr_matrix'], True, "QED_FSR", 0)

        unfoldClass.setSysFSRTUnfoldDensity("Pt",   unfoldInputList['fsr_pythia_matrix'], "QED_FSR", 2, 1, phase_space_fsr, FSR_dR_fsr)
        unfoldClass.setSysFSRTUnfoldDensity("Mass", unfoldInputList['fsr_pythia_matrix'], "QED_FSR", 2, 1, phase_space_fsr, FSR_dR_fsr)

        unfoldClass.setFSRUnfoldInput(unfoldInputList['fsr_matrix'], True, "QED_FSR", 1)

    # unfolding for QED FSR
    unfoldClass.doISRUnfold(FSRUnfold, doSys)

    # set nominal value and also systematic values
    unfoldClass.setMeanPt(doSys, False, doSys)
    unfoldClass.setMeanMass(doSys, False, doSys)

    # NOW DRAW PLOTS
    dirClosurePlots = "ClosurePlots/"
    if not os.path.exists( outputDirectory + dirClosurePlots ):
        os.makedirs( outputDirectory + dirClosurePlots )

    dirUnfoldHists = "UnfoldHist/"
    if not os.path.exists( outputDirectory + dirUnfoldHists ):
        os.makedirs( outputDirectory + dirUnfoldHists )

    unfoldClass.drawNominalRecoPlots(outputDirectory + "Reco_" + channel , unfoldInputList['hist'], "Mass", 5)

    for massBin in range(0,5):
        # closure test
        unfoldClass.drawClosurePlots(DetectorUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "DetClosure_" + channel, "Pt", massBin)
        unfoldClass.drawClosurePlots(DetectorUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "DetClosure_" + channel, "Mass", massBin)
        unfoldClass.drawClosurePlots(FSRUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "FSRClosure_" + channel, "Pt", massBin)
        unfoldClass.drawClosurePlots(FSRUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "FSRClosure_" + channel, "Mass", massBin)

        unfoldClass.drawNominalRecoPlots(outputDirectory + "Reco_" + channel , unfoldInputList['hist'], "Pt", massBin)


        if not doSys:
            # detector unfold
            unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel, "Pt", massBin)
            unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel, "Mass", massBin)

            # FSR unfold
            unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel, "Pt",   massBin, "", False, True)
            unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel, "Mass", massBin, "", False, True)

    unfoldClass.drawClosurePlots(DetectorUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "DetClosure_" + channel, "Pt", 5)
    unfoldClass.drawClosurePlots(FSRUnfold, unfoldInputList['gen_hist_closure'], outputDirectory + dirClosurePlots + "FSRClosure_" + channel, "Pt", 5)

    # draw plots including systematic
    if doSys == True:
        for sysName, nSys in sysDict.items():

            for massBin in range(0,5):

                unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel+sysName, "Pt", massBin, sysName, doSys)
                unfoldClass.drawUnfoldedHists(outputDirectory + dirUnfoldHists + "Unfolded_"+channel+sysName, "Mass", massBin, sysName, doSys)
                unfoldClass.drawSysPlots(outputDirectory + "Sys_" + channel , massBin, sysName, True)
                unfoldClass.drawSysPlots(outputDirectory + "Sys_" + channel , massBin, sysName, False)

                unfoldClass.drawSysPlots(outputDirectory + "Sys_" + channel , massBin, "full", True)
                unfoldClass.drawSysPlots(outputDirectory + "Sys_" + channel , massBin, "full", False)

                unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ channel+"fullSys", "Pt", massBin, "full", doSys, True)
                #unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ args.channel+"fullSys", "Pt", massBin, "full", args.doSys, True, True) # normalized with area
                unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ channel+"fullSys", "Mass", massBin, "full", doSys, True)

                if sysName is not "QED_FSR":
                    # no QED_FSR systematic variabtion on detector unfolding
                    unfoldClass.drawSysComparionPlots(outputDirectory + "Sys_" + channel , "Pt", massBin, sysName, True)
                    unfoldClass.drawInputPlots(outputDirectory + channel + sysName, "Pt", massBin, sysName)

                unfoldClass.drawSysComparionPlots(outputDirectory + "Sys_" + channel , "Pt", massBin, sysName, False)

            if sysName is not "QED_FSR":
                unfoldClass.drawSysComparionPlots(outputDirectory + "Sys_" + channel , "Mass", massBin, sysName, True)
            unfoldClass.drawSysComparionPlots(outputDirectory + "Sys_" + channel , "Mass", massBin, sysName, False)
        
        unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ channel+"fullSys", "Mass", 5, "full", doSys, True)
        unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ channel+"fullSys", "Pt", 5, "full", doSys, True)
        #unfoldClass.drawUnfoldedHists(outputDirectory + "Unfolded_"+ args.channel+"fullSys", "Pt", 5, "full", args.doSys, True, True)


    unfoldClass.drawISRMatrixInfo("Pt", outputDirectory, True)
    unfoldClass.drawISRMatrixInfo("Pt", outputDirectory, False)

    unfoldClass.drawISRMatrixInfo("Mass", outputDirectory, True)
    unfoldClass.drawISRMatrixInfo("Mass", outputDirectory, False)

    unfoldClass.drawSysSummaryPlots(outputDirectory + "ISRSysSummary_", False)
    unfoldClass.drawISRresult(outputDirectory + "ISRfit_", True, "output/2016/" + channel + "/")

    #unfoldClass.drawISRMatrixInfo("Pt", outputDirectory, False, True)
    #unfoldClass.drawISRMatrixInfo("Mass", outputDirectory, False, True)

    del unfoldClass

In [4]:
doISRAnalysis("2016", "electron", True)

{'matrix': 'output/2016/electron/DY_v3.root', 'matrix_pdferr': 'output/2016/electron/DY_PDF.root', 'matrix_closure': 'output/2016/electron/ISR_response_closureTest.root', 'fsr_matrix_closure': 'output/2016/electron/ISR_response_FSR_closureTest.root', 'matrix_alt': 'output/2016/electron/DY_MG.root', 'fsr_matrix': 'output/2016/electron/DY_FSR.root', 'fsr_matrix_alt': 'output/2016/electron/DY_FSR_powheg.root', 'fsr_photos_matrix': 'output/2016/electron/DY_FSR_photos_v3.root', 'fsr_pythia_matrix': 'output/2016/electron/DY_FSR_pythia_v3.root', 'gen_hist': 'output/2016/electron/DY_Gen_Hists.root', 'gen_hist_closure': 'output/2016/electron/ISR_Gen_Hists_closureTest.root', 'hist': 'output/2016/electron/unfolding_input_v2.root'}
sysName: PU nthSys: 0 #####################################################
sysName: PU nthSys: 1 #####################################################
sysName: trgSF nthSys: 0 #####################################################
sysName: trgSF nthSys: 1 ##############

sysName: Stat nthSys: 331 #####################################################
sysName: Stat nthSys: 332 #####################################################
sysName: Stat nthSys: 333 #####################################################
sysName: Stat nthSys: 334 #####################################################
sysName: Stat nthSys: 335 #####################################################
sysName: Stat nthSys: 336 #####################################################
sysName: Stat nthSys: 337 #####################################################
sysName: Stat nthSys: 338 #####################################################
sysName: Stat nthSys: 339 #####################################################
sysName: Stat nthSys: 340 #####################################################
sysName: Stat nthSys: 341 #####################################################
sysName: Stat nthSys: 342 #####################################################
sysName: Stat nthSys: 343 ##############

i: 0 detector_level/hist_ptll/histo_DYJetsnominal
i: 0 detector_level_DY_Fake/hist_ptll/histo_DYJetsnominal
i: 1 detector_level_unfold_split_p5/hist_ptll/histo_DYJetsnominal
i: 1 detector_level_DY_Fake_unfold_split_p5/hist_ptll/histo_DYJetsnominal
i: 2 detector_level_unfold_split_p2/hist_ptll/histo_DYJetsnominal
i: 2 detector_level_DY_Fake_unfold_split_p2/hist_ptll/histo_DYJetsnominal
histo: detector_level/hist_ptll/histo_DYJetsToTauTaunominal
histo: detector_level/hist_ptll/histo_DYJets10to50ToTauTaunominal
histo: detector_level/hist_ptll/histo_TTLL_powhegnominal
histo: detector_level/hist_ptll/histo_WW_pythianominal
histo: detector_level/hist_ptll/histo_WZ_pythianominal
histo: detector_level/hist_ptll/histo_ZZ_pythianominal
histo: detector_level/hist_ptll/histo_WJets_MGnominal
histo: detector_level_DY_Fake/hist_ptll/histo_DYJetsnominal
histo: detector_level_DY_Fake/hist_ptll/histo_DYJets10to50nominal
path: detector_level/hist_ptll/histo_DYJetsToTauTau_PUUp
path: detector_level/hist_p

Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 126 input bins and 70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 126 input bins and 70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnfoldV17::SetConstraint>: fConstraint=1
Info in <TUnfoldV17::TUnfold>: underflow and overflow bin do not depend on the input data
Info in <TUnfoldV17::TUnfold>: 126 input bins and 70 output bins
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #0
Info in <TUnfoldDensityV17::TUnfold>: *NOT* unfolding bin #71
Info in <TUnf